In [1]:

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
matplotlib.use('Qt5Agg') 
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from qutip import *

import src.encoding.evolution as evolution
import importlib
importlib.reload(evolution)
from src.encoding.evolution import Evolve
from tqdm import tqdm
from src.helper_functions import *
from src.encoding.classify import Classify


In [3]:
M = 2*np.eye(3)

In [1]:
import dynamiqs as dq
import jax.numpy as jnp

In [5]:

result = dq.tensor(dq.fock_dm(2, 0), jnp.array([[dq.fock_dm(10, 1)], [dq.fock_dm(10, 2)]]))

In [8]:
disps = jnp.array([dq.displace(10, 1), dq.displace(10,2)])

In [10]:
dq.dag(disps).shape

(2, 10, 10)

In [7]:
xvec = np.linspace(-3, 3, 100)
yvec = np.linspace(-3, 3, 100)
W = wigner(fock_dm(10, 1), xvec, yvec)
fig, axes = plt.subplots(1, 1, figsize=(10,10))
axes.contourf(xvec, yvec, W)
plt.show()

In [3]:
def linspace(start, stop, step=1.):
  """
    Like np.linspace but uses step instead of num
    This is inclusive to stop, so if start=1, stop=3, step=0.5
    Output is: array([1., 1.5, 2., 2.5, 3.])
  """
  return np.linspace(start, stop, int((stop - start) / step + 1))


In [5]:
cdim = 20
qdim = 2

chi = 1.467
#kerr = 3e-3


In [6]:
a = tensor(qeye(qdim), destroy(cdim))
ad = a.dag()

q = tensor(destroy(qdim), qeye(cdim))
qd = q.dag()
X = tensor(qeye(qdim), position(cdim))
P = tensor(qeye(qdim), momentum(cdim))

In [193]:
kerrs = linspace(1e-3, 5e-3, 0.5e-3)
coeffs = []
for kerr in kerrs:
    H =   - np.pi * kerr * ad * ad * a * a
    ns = linspace(1, 1, 0.1)
    tlist = linspace(0, 15, 0.1)
    grads = []
    for n in ns:
        rho0 = tensor(fock_dm(qdim, 0), coherent_dm(cdim, n))
        result = mesolve(H, rho0, tlist=tlist).states
        ms = [((r*X).tr().real/np.sqrt(2) - n, (r*P).tr().real) for r in result]
        angles = [np.arctan2(*m[::-1]) for m in ms]
        angles[0] = np.pi/2
        grads.append(np.polyfit(tlist, angles ,1)[0])
    coeffs.append(np.polyfit(ns, grads, 1)[0])

In [194]:
plt.plot(kerrs, coeffs)
plt.show()

In [9]:
kerr = 3e-3
H =   - np.pi * kerr * ad * ad * a * a
t = 15
step = 0.1
ns = linspace(1, 1, 0.1)
tlist = linspace(0, t, step)
rho0 = tensor(fock_dm(qdim, 0), coherent_dm(cdim, 1))
result = mesolve(H, rho0, tlist=tlist).states
ms = [((r*X).tr().real/np.sqrt(2) - 1, (r*P).tr().real) for r in result]
angles = [np.arctan2(*m[::-1]) for m in ms]
angles[0] = np.pi/2
scale = 4
xvec = np.linspace(-scale+1, scale+1, 100)
yvec = np.linspace(-scale, scale, 100)
Ws = np.array([wigner(r, xvec, yvec) for r in result])

In [10]:

# Define initial parameters

# Create the figure and the line that we will manipulate
fig, ax = plt.subplots(figsize=(10, 10))

ax.contourf(xvec/np.sqrt(2)-1, yvec/np.sqrt(2), Ws[0], vmin=-0.2, vmax=0.6)
p, = ax.plot(*ms[0], 'ro')
angle = 0
# adjust the main plot to make room for the sliders
fig.subplots_adjust(left=0.25, bottom=0.25)
ax.set_title(f"angle: {angle}")
# Make a horizontal slider to control the frequency.
axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='Time [µs]',
    valmin=0,
    valmax=t,
    valinit=0,
    valstep=step
)


# The function to be called anytime a slider's value changes
def update(val):
    index = int(np.rint(freq_slider.val/step))
    ax.contourf(xvec/np.sqrt(2)-1, yvec/np.sqrt(2), Ws[index], vmin=-0.2, vmax=0.6)
    p.set_xdata([ms[index][0]])
    p.set_ydata([ms[index][1]])
    angle = angles[index]
    ax.set_title(f"angle: {angle}")


# register the update function with each slider
freq_slider.on_changed(update)

plt.show()